In [20]:
# モーフィングの補題を実装してみる
import numpy as np

def span_includes(A, v, debug=False):
    rank_a  = np.linalg.matrix_rank(np.array(A).transpose())
    rank_ab = np.linalg.matrix_rank(np.array(A + [v]).transpose())
    
    if debug:
        print(rank_a, rank_ab)

    return rank_a == rank_ab

def morph_S(S, B):
    '''
    S: 生成子の集合
    B: Span(S) 内の線形独立なベクトル
    '''
    Sc = np.array(S)
    A = []
    for b in B:
        A.append(b) # B から S に１個取り込む（その後取り除かれないよう、S の部分集合 A として区別する）
        n = len(Sc)
        for i in range(len(Sc)):
            pick = [True] * n
            pick[i] = False
            T = Sc[pick].tolist() + A
            w = Sc[i].tolist()
            if span_includes(T, w):
                # w は S から削除可能
                Sc = Sc[pick]
                break
    return Sc.tolist() + A

In [17]:
S = [
    [1,0,0],
    [0,1,0],
    [0,0,1]
]
B1 = [
    [1,2,3],
    [4,5,6]
]

morph_S(S, B1)

[[0, 0, 1], [1, 2, 3], [4, 5, 6]]

In [18]:
B2 = [
    [0,3,2],
    [0,4,1]
]
morph_S(S, B2)

[[1, 0, 0], [0, 3, 2], [0, 4, 1]]

In [19]:
B3 = [
    [3,0,0],
    [0,5,0],
    [0,0,8]
]
morph_S(S, B3)

[[3, 0, 0], [0, 5, 0], [0, 0, 8]]

In [21]:
B4 = [
    [0,1,0],
    [0,0,1],
    [1,0,0]
]
morph_S(S, B4)

[[0, 1, 0], [0, 0, 1], [1, 0, 0]]

In [1]:
from vecutil import *
from independence import *
from GF2 import one

In [14]:
# p342 問題 6.3.13, 6.3.14

def check_rank(L):
    return rank([list2vec(l) for l in L])

# すべてランクが 3 であれば正解
print(check_rank([[2,1,2], [1,1,1], [1,1,0]]))
print(check_rank([[0,1,-1], [1,0,0], [0,1,0]]))

print(check_rank([[one,one,0], [0,one,one], [0,0,one]]))
print(check_rank([[one,one,one], [0,one,0], [0,0,one]]))

3
3
3
3


In [8]:
from matutil import *

def is_invertible(M):
    if len(M.D[0]) != len(M.D[1]):
        return False
    return is_independent(list(mat2coldict(M).values()))

M = Mat(({0, 1, 2, 3}, {0, 1, 2, 3}), {(0, 1): 0, (1, 2): 1, (3, 2): 0, (0, 0): 1, (3, 3): 4, (3, 0): 0, (3, 1): 0, (1, 1): 2, (2, 1): 0, (0, 2): 1, (2, 0): 0, (1, 3): 0, (2, 3): 1, (2, 2): 3, (1, 0): 0, (0, 3): 0})
is_invertible(M)

M1 = Mat(({0,1,2},{0,1,2}),{(0,0):1,(0,2):2,(1,2):3,(2,2):4})
is_invertible(M1)

False

In [12]:
from solver import *

def find_matrix_inverse(A):
    '''
    Input:
        - A: an invertible Mat over GF(2)
    Output:
        - A Mat that is the inverse of A
    Examples:
        >>> M1 = Mat(({0,1,2}, {0,1,2}), {(0, 1): one, (1, 0): one, (2, 2): one})
        >>> find_matrix_inverse(M1) == Mat(M1.D, {(0, 1): one, (1, 0): one, (2, 2): one})
        True
    '''
    x = []
    n = len(A.D[0])
    for i in range(n):
        bi = list2vec([one if j == i else 0 for j in range(n)])
        x.append(solve(A, bi))
    return coldict2mat(x)

M1 = Mat(({0,1,2}, {0,1,2}), {(0, 1): one, (1, 0): one, (2, 2): one})
find_matrix_inverse(M1) == Mat(M1.D, {(0, 1): one, (1, 0): one, (2, 2): one})

True

In [28]:
from triangular import triangular_solve

def find_triangular_matrix_inverse(A):
    x = []
    n = len(A.D[0])
    for i in range(n):
        bi = list2vec([1 if i == j else 0 for j in range(n)])
        x.append(triangular_solve(rowlist, range(n), bi))
    return coldict2mat(x)

A = listlist2mat([[1, .5, .2, 4],[0, 1, .3, .9],[0,0,1,.1],[0,0,0,1]])
A2 = find_triangular_matrix_inverse(A)# == Mat(({0, 1, 2, 3}, {0, 1, 2, 3}), {(0, 1): -0.5, (1, 2): -0.3, (3, 2): 0.0, (0, 0): 1.0, (3, 3): 1.0, (3, 0): 0.0, (3, 1): 0.0, (2, 1): 0.0, (0, 2): -0.05000000000000002, (2, 0): 0.0, (1, 3): -0.87, (2, 3): -0.1, (2, 2): 1.0, (1, 0): 0.0, (0, 3): -3.545, (1, 1): 1.0})
print(A)
print(A2)
print(Mat(({0, 1, 2, 3}, {0, 1, 2, 3}), {(0, 1): -0.5, (1, 2): -0.3, (3, 2): 0.0, (0, 0): 1.0, (3, 3): 1.0, (3, 0): 0.0, (3, 1): 0.0, (2, 1): 0.0, (0, 2): -0.05000000000000002, (2, 0): 0.0, (1, 3): -0.87, (2, 3): -0.1, (2, 2): 1.0, (1, 0): 0.0, (0, 3): -3.545, (1, 1): 1.0}))
print(A * A2)


       0   1   2   3
     ---------------
 0  |  1 0.5 0.2   4
 1  |  0   1 0.3 0.9
 2  |  0   0   1 0.1
 3  |  0   0   0   1


       0    1     2     3
     --------------------
 0  |  1 -0.5 -0.05 -3.54
 1  |  0    1  -0.3 -0.87
 2  |  0    0     1  -0.1
 3  |  0    0     0     1


       0    1     2     3
     --------------------
 0  |  1 -0.5 -0.05 -3.54
 1  |  0    1  -0.3 -0.87
 2  |  0    0     1  -0.1
 3  |  0    0     0     1


       0 1 2 3
     ---------
 0  |  1 0 0 0
 1  |  0 1 0 0
 2  |  0 0 1 0
 3  |  0 0 0 1

